# AbhiBus Data Engineering and Feature Selection

I will be performing Data Engineering work to the following column present in the FE- Ticketnig Data.

# Importing the Dataset from Google Drive

In [0]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.4MB/s 


In [3]:
# inseting the sharable link into "link" variable
link = 'https://drive.google.com/open?id=1GBbtiSKnq4cmRIIbi7r6fMSVKRQJkDSk'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1GBbtiSKnq4cmRIIbi7r6fMSVKRQJkDSk


In [4]:
# Fecthing the dataset from Google Drive
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
columns = ['PNR','Source','Destination','DOJ','OperatorName','BusType','ServiceType','BoardingPoint','BoardingTime','DroppoingPoint',
          'DroppingTime','Email','Mobile','Age','Gender','Insurance','insurance_amount','CouponCode','Discount_Amt','WalletAmountRedemeed',
          'PGAmount','PGType','GSTAmount','ServiceCharge','TicketAmount','No_Of_Seats','BookingPlatform','BookedDate','BookingTime',
          'CancellationDate','CancellationTime','CancelledSeats','CancellationAmount','RefundGSTAmount','CustomerId','EdgeOfferAmount',
          'APISource','Abhiid','Earnings','ReferenceNo']

dataset = pd.read_csv('Filename.csv', names = columns)
print(dataset.head(5))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


             PNR     Source  Destination  ... Abhiid Earnings ReferenceNo
0     DWK1260604  Tadipatri    Hyderabad  ...      6        0         NaN
1     DWK1261003  Hyderabad    Tadipatri  ...      6        0         NaN
2     DWK1262071  Hyderabad    Tadipatri  ...      6        0         NaN
3  KAVERI2306276  Hyderabad  Ravulapalem  ...      6        0         NaN
4  KAVERI2313805  Hyderabad  Ganapavaram  ...      6        0         NaN

[5 rows x 40 columns]


# Dataset Observation


In [5]:
# Observing the Dataset 
data = dataset

# Shape of Dataset
print("Shape of Dataset is : ", data.shape)
print("\n")
# Features of Dataset
print("Featues/Columns in Dataset :", data.columns)
print("\n")
# Number of Features in Dataset
print("Number of Feature/Column in the Dataset :", len(dataset.columns))

Shape of Dataset is :  (1012722, 40)


Featues/Columns in Dataset : Index(['PNR', 'Source', 'Destination', 'DOJ', 'OperatorName', 'BusType',
       'ServiceType', 'BoardingPoint', 'BoardingTime', 'DroppoingPoint',
       'DroppingTime', 'Email', 'Mobile', 'Age', 'Gender', 'Insurance',
       'insurance_amount', 'CouponCode', 'Discount_Amt',
       'WalletAmountRedemeed', 'PGAmount', 'PGType', 'GSTAmount',
       'ServiceCharge', 'TicketAmount', 'No_Of_Seats', 'BookingPlatform',
       'BookedDate', 'BookingTime', 'CancellationDate', 'CancellationTime',
       'CancelledSeats', 'CancellationAmount', 'RefundGSTAmount', 'CustomerId',
       'EdgeOfferAmount', 'APISource', 'Abhiid', 'Earnings', 'ReferenceNo'],
      dtype='object')


Number of Feature/Column in the Dataset : 40


# Data Engineering on the DOJ

In [6]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1rxq1QZrPYSeZmp7aQzhCRaAiCRZj4UxV">')

**Calculating the Day of the Week**

In [7]:
# Verifying the Null/Nan present in the DOJ column
data["DOJ"].isna().sum()

0

In [0]:
# Function to get the Day of the week
Day_Of_Week = []

def weekDay(year, month, day):
    offset = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
    week   = ['Sunday', 
              'Monday', 
              'Tuesday', 
              'Wednesday', 
              'Thursday',  
              'Friday', 
              'Saturday']
    afterFeb = 1
    if month > 2: afterFeb = 0
    aux = year - 1700 - afterFeb
    # dayOfWeek for 1700/1/1 = 5, Friday
    dayOfWeek  = 5
    # partial sum of days betweem current date and 1700/1/1
    dayOfWeek += (aux + afterFeb) * 365                  
    # leap year correction    
    dayOfWeek += aux / 4 - aux / 100 + (aux + 100) / 400     
    # sum monthly and day offsets
    dayOfWeek += offset[month - 1] + (day - 1)               
    dayOfWeek %= 7
    Day_Of_Week.append(int(dayOfWeek))



In [0]:
# sending the date to weekDay function
for days in data.DOJ:
    year, month, day = (int(x) for x in days.split('-'))  
    weekDay(year, month, day)

In [10]:
print(Day_Of_Week[:10])

[1, 4, 6, 0, 0, 5, 1, 4, 5, 6]


**Putting WeekEnd flag on Friday and Sunday Travel**


In [0]:
WeekEndFlag = []
"""
if WeekEndFlag has
0 then non-Weekend journey
1 then weekend journey

"""
for day in Day_Of_Week:
    if day == 0 or day == 5:
        WeekEndFlag.append("1")
    else:
        WeekEndFlag.append("0")


In [12]:
print(len(WeekEndFlag))
print(WeekEndFlag[:10])

1012722
['0', '0', '0', '1', '1', '1', '0', '0', '1', '0']


***Result of Data Engineering on DOJ in DataFrame format***

In [13]:
# Creating a data frame with result obtained from the DOJ
DOJ_Result = pd.DataFrame(list(zip(Day_Of_Week, WeekEndFlag)), 
               columns =['Day_Of_Week', 'Week_End_Flag']) 

print(DOJ_Result.head(5))

   Day_Of_Week Week_End_Flag
0            1             0
1            4             0
2            6             0
3            0             1
4            0             1


# Data Engineering on OperatorName

In [14]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1_WGZtLnlzUWr_qncoGZ6zHuAVBFpjNdl">')


**Take the ratings of the Operator and make it ordinal variable.**

***No Ratings column present in the dataset. Hence, creation of Ordinal Variable is on-hold***

**Classifying the Operator whether private(1) or Public(0) and creating the list with Binary Values**

In [15]:
# Validating whether the any RTC are present dataset
print(data["OperatorName"].str.find("APSRTC").sum()) 


-1012722


In [0]:
# Checking the APSRTC, KSRTC  or any RTC data present in the dataset
# Since no RTC has been found in dataset proceeding further to create a list with Public/Private Operator
count = 0
RTC_list = ["APSRTC","KSRTC"]
Operator_Class = []
for operator in data.OperatorName:
  if operator in RTC_list:
    Operator_Class.append("0")
  else:
    Operator_Class.append("1")
    count += 1

***Result for the OperatorName column***

In [17]:
# Creating the Result DataFrame for the OperatorName
OperatorName_Result = pd.DataFrame(Operator_Class, columns =['Operator_Class'])
print(OperatorName_Result.head(5))

  Operator_Class
0              1
1              1
2              1
3              1
4              1


# Data Engineering on BusType

In [18]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1vMK7sJPXB5ZfFaSHwqoTy986LB9FZJ-s">')


In [19]:
# Counting the unique categorical data present in the 
cat_data = data.select_dtypes(include=['object']).copy()
print(cat_data.columns)

Index(['PNR', 'Source', 'Destination', 'DOJ', 'OperatorName', 'BusType',
       'ServiceType', 'BoardingPoint', 'BoardingTime', 'DroppoingPoint',
       'DroppingTime', 'Email', 'Mobile', 'Age', 'Gender', 'Insurance',
       'CouponCode', 'PGType', 'BookingPlatform', 'BookedDate', 'BookingTime',
       'CancellationDate', 'CancellationTime', 'CustomerId', 'APISource',
       'ReferenceNo'],
      dtype='object')


In [20]:
print(data['BusType'].value_counts())

Hi-Tech Non AC                                          239104
Volvo B9R Multi-Axle Semi Sleeper                       152505
Volvo AC B9R Multi-Axle Semi Sleeper                     70464
Non AC Seater 2+2                                        35524
Multiaxle B9R VOLVO AC Semi Sleeper                      30291
AC Sleeper                                               26094
Volvo B11R Multi-Axle Semi Sleeper Auto Transmission     23617
Brand New Volvo AC B9R Multi-Axle Semi Sleeper           21307
Hi-Tech Non AC 2+2 Seater                                19105
Volvo Semi Sleeper                                       18279
Non AC Seater                                            16017
VOLVO                                                    15621
Hitech Non AC                                            15447
AC CLASSY                                                14575
Volvo AC                                                 14350
SLEEPER CUM SEMI SLEEPER(NON AC)                       

In [21]:
print(data["ServiceType"].head(10))

0    NON A/C
1    NON A/C
2    NON A/C
3        A/C
4    NON A/C
5    NON A/C
6        A/C
7    NON A/C
8    NON A/C
9        A/C
Name: ServiceType, dtype: object


*** Applying Kmode to OperatorName,BusType, ServiceType and TicketAmount***

In [22]:
# Creating the Dataframe to apply the K-Mode and extract the cluster hidden for the BusType, OperatorName,ServiceType and TicketAmount
BT_df = pd.DataFrame(list(zip(data.BusType,data.TicketAmount, data.ServiceType,  data.OperatorName)), 
               columns =['BusType',  'TicketAmount', 'ServiceType','OperatorName']) 

print(BT_df.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012722 entries, 0 to 1012721
Data columns (total 4 columns):
BusType         1012722 non-null object
TicketAmount    1012722 non-null float64
ServiceType     1012722 non-null object
OperatorName    1012722 non-null object
dtypes: float64(1), object(3)
memory usage: 30.9+ MB
None


In [23]:
!pip install kModes
import numpy as np
from kmodes.kprototypes import KPrototypes

# stocks with their market caps, sectors and countries
syms = BT_df.iloc[:,0]
syms = syms.head(10000)
X = BT_df.iloc[:, [1,2,3]]
X = X.head(10000)


In [24]:
# Looking at top 10000 records of BT_DF
print("Description")
print(X.describe())
print("\n")
print("Information")
print(X.info())

Description
       TicketAmount
count  10000.000000
mean     957.654720
std     1150.746456
min        0.000000
25%      350.000000
50%      699.000000
75%     1220.000000
max    19920.000000


Information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
TicketAmount    10000 non-null float64
ServiceType     10000 non-null object
OperatorName    10000 non-null object
dtypes: float64(1), object(2)
memory usage: 234.5+ KB
None


In [25]:
print("Standard Ticket Amount Value=", X["TicketAmount"].std())
print("Mean Ticket Amount Value=",X["TicketAmount"].mean())
Avg_TicketAmt = X["TicketAmount"].mean()

Standard Ticket Amount Value= 1150.7464564544287
Mean Ticket Amount Value= 957.6547200000001


In [26]:

X["TicketAmount"] = X["TicketAmount"].astype(float)

kproto = KPrototypes(n_clusters=7, init='Cao', verbose=2)
clusters = kproto.fit_predict(X, categorical=[1, 2])

# Print cluster centroids of the trained model.
print(kproto.cluster_centroids_)
# Print training statistics
print(kproto.cost_)
print(kproto.n_iter_)

for s, c in zip(syms, clusters):
    print("Symbol: {}, cluster:{}".format(s, c))


Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 1595, ncost: 1400578443.1488595
Run: 1, iteration: 2/100, moves: 555, ncost: 1223587421.1629663
Run: 1, iteration: 3/100, moves: 234, ncost: 1171972092.0298586
Run: 1, iteration: 4/100, moves: 347, ncost: 1106510097.9857364
Run: 1, iteration: 5/100, moves: 443, ncost: 1092840489.317563
Run: 1, iteration: 6/100, moves: 253, ncost: 1008101359.4123976
Run: 1, iteration: 7/100, moves: 172, ncost: 927252781.1581174
Run: 1, iteration: 8/100, moves: 119, ncost: 896036298.9409461
Run: 1, iteration: 9/100, moves: 273, ncost: 828696687.5070496
Run: 1, iteration: 10/100, moves: 470, ncost: 767508580.173238
Run: 1, iteration: 11/100, moves: 151, ncost: 757276112.6582487
Run: 1, iteration: 12/100, moves: 296, ncost: 730523928.9632846
Run: 1, iteration: 13/100, moves: 294, ncost: 722242312.5302608
Run: 1, iteration: 14/100, moves: 5

***Result for BusType Data Engineering.***

In [27]:
# Creating the DF for BusType 
BusType_result = pd.DataFrame(list(zip(syms,clusters)), columns =['BusType',"BusType_Cluster"])
print("BusType Data Engineering Results")
print(BusType_result.head(10))


BusType Data Engineering Results
                             BusType  BusType_Cluster
0           Semi Sleeper cum Sleeper                6
1           Semi Sleeper cum Sleeper                6
2           Semi Sleeper cum Sleeper                6
3  Volvo B9R Multi-Axle Semi Sleeper                0
4                     Hi-Tech Non AC                0
5                  Non AC Seater 2+2                0
6                           Volvo AC                6
7           Semi Sleeper cum Sleeper                6
8                  Non AC Seater 2+2                0
9  Volvo B9R Multi-Axle Semi Sleeper                0


# Data Engineering on BoardingTime

In [28]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1FJjsM_hmhe1ZKYdFwgRFFMTGtOtleDdH">')


In [29]:
# Verifying the Null/Nan present in the BoardingTime column
data["BoardingTime"].isna().sum()

0

In [30]:
BoardingTime = data["BoardingTime"]
print(BoardingTime.head(5))

0    09:30 PM
1    11:00 PM
2    11:00 PM
3    09:50 PM
4    11:25 PM
Name: BoardingTime, dtype: object


In [0]:
# Python program to convert time 
# from 12 hour to 24 hour format 

# Function to convert the date format 
def convert24(str1): 
	
	# Checking if last two elements of time 
	# is AM and first two elements are 12 
	if str1[-2:] == "AM" and str1[:2] == "12": 
		return "00" + str1[2:-2] 
		
	# remove the AM	 
	elif str1[-2:] == "AM": 
		return str1[:-2] 
	
	# Checking if last two elements of time 
	# is PM and first two elements are 12 
	elif str1[-2:] == "PM" and str1[:2] == "12": 
		return str1[:-2] 
		
	else: 
		
		# add 12 to hours and remove PM 
		return str(int(str1[:2]) + 12) + str1[2:8] 



In [0]:
BT_format = []
for time in BoardingTime.head(100):
  if "\\N" in time:
     BT_format.append("NULL")
  else:
    times = time[:5] + ":00 " + time[-2:]
    BT_format.append(convert24(times))



In [33]:
print(BT_format)

['21:30:00', '23:00:00', '23:00:00', '21:50:00', '23:25:00', '21:20:00', '22:30:00', '22:40:00', '21:20:00', '18:00:00', '21:30:00', '22:15:00', 'NULL', '22:07:00', '19:10:00', '21:30:00', '23:00:00', '18:45:00', '20:50:00', '16:00:00', '17:15:00', '17:15:00', '22:30:00', '19:20:00', '20:00:00', '22:35:00', '22:35:00', '21:00:00', '22:00:00', '20:45:00', '22:00:00', '22:00:00', '21:45:00', 'NULL', '23:00:00', '22:35:00', '22:35:00', '22:40:00', '23:20:00', '22:40:00', '22:35:00', '21:45:00', '14:00:00', '22:50:00', '22:15:00', '23:00:00', '23:00:00', '23:59:00', '20:45:00', '21:45:00', 'NULL', '22:00:00', '22:50:00', '23:30:00', '23:00:00', '22:30:00', '22:15:00', '23:15:00', '20:45:00', '22:00:00', '22:30:00', '21:55:00', '21:46:00', '18:30:00', '20:30:00', '20:30:00', '20:15:00', '20:15:00', '19:45:00', '20:15:00', '21:00:00', '21:45:00', '17:15:00', '21:15:00', '20:15:00', '20:00:00', '20:00:00', '21:00:00', '21:55:00', '23:00:00', '23:20:00', '23:59:00', '23:20:00', '20:30:00', '23

In [0]:
# Here we can change timing class as per the Indian Timings Schedule

from sys import stdout

def get_part_of_day(hour):
    return (
        "morning" if 5 <= hour <= 11
        else
        "afternoon" if 12 <= hour <= 17
        else
        "evening" if 18 <= hour <= 22
        else
        "night"
    )


In [35]:
BT_class = []
for typo in BT_format:
  if typo =="NULL":
    BT_class.append("NULL")
  else:
    BT_class.append(str(get_part_of_day(int(typo[:2]))))

print(BT_class)

['evening', 'night', 'night', 'evening', 'night', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'NULL', 'evening', 'evening', 'evening', 'night', 'evening', 'evening', 'afternoon', 'afternoon', 'afternoon', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'NULL', 'night', 'evening', 'evening', 'evening', 'night', 'evening', 'evening', 'evening', 'afternoon', 'evening', 'evening', 'night', 'night', 'night', 'evening', 'evening', 'NULL', 'evening', 'evening', 'night', 'night', 'evening', 'evening', 'night', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'afternoon', 'evening', 'evening', 'evening', 'evening', 'evening', 'evening', 'night', 'night', 'night', 'night', 'evening', 'night', 'evening', 'evening', 'evening', 'NULL', 'evening', 'night', 'evening', 'evening', 'NULL', 'NULL'

In [36]:
BoardingTime_class = pd.DataFrame(zip(data["BoardingTime"].head(100),BT_class), columns =['BoardingTime',"BoardingTimeClass"])
print(BoardingTime_class.head(5))

  BoardingTime BoardingTimeClass
0     09:30 PM           evening
1     11:00 PM             night
2     11:00 PM             night
3     09:50 PM           evening
4     11:25 PM             night


***Result of Data Engineering for BoardingTime( only 100 line is considered)***

In [37]:
BoardingTime_result = pd.DataFrame(BT_class, columns=['BoardingTime_Class'])
print(BoardingTime_result.head(5))

  BoardingTime_Class
0            evening
1              night
2              night
3            evening
4              night


# Data Engineering on DroppingTime

In [38]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1nOA-l2TfPdcG3oxlFF10ECZ10ecRqVfe">')


***Note: We need to have Date Of Journey and Date of Completion Journey to have valid result for Time of Journey. Since we dont have Data for date of completion of Journey ,I will assume Journey timings are not greater than 1 Day i.e 24hours***

In [0]:
#Dropping Time of dataset
DroppingTime = data["DroppingTime"]

#converting 12hr to 24hr format
DT_format = []
for time in DroppingTime.head(100):
  if "\\N" in time:
     DT_format.append("NULL")
  else:
    times = time[:5] + ":00 " + time[-2:]
    DT_format.append(convert24(times))



In [40]:
print(DT_format)

['17:30:00', '18:30:00', '18:30:00', '18:00:00', '17:30:00', '18:00:00', 'NULL', '18:30:00', '16:20:00', '17:30:00', '21:00:00', '18:00:00', '19:45:00', '17:30:00', '20:30:00', '17:30:00', '18:30:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '21:00:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '20:00:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '20:00:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '18:30:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '20:30:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '21:00:00', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '16:30:00', '16:30:00', '16:30:00', '16:30:00', 'NULL', '16:30:00', 'NULL']


In [41]:
# Calculating the total Time of Journey.
TOJ = []
for BT, DT in zip(BT_format, DT_format):
  
  if BT == "NULL" or DT == "NULL":
    if BT == "NULL":
      TOJ.append("BT_NULL")   
    if DT == "NULL":
      TOJ.append("DT_NULL")   
  else:
    val1 = BT[:2]
    BT_diff = abs(24 - int(val1))
    val2 = DT[:2]
    DT_diff = abs(0 + int(val2))
    TOJ.append(BT_diff + DT_diff)

print(TOJ)

[20, 19, 19, 21, 18, 21, 'DT_NULL', 20, 19, 23, 24, 20, 'BT_NULL', 19, 25, 20, 19, 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 23, 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 24, 'DT_NULL', 'DT_NULL', 'DT_NULL', 'BT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'BT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 20, 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 23, 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 22, 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'BT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'DT_NULL', 'BT_NULL', 'BT_NULL', 'BT_NULL', 'BT_NULL', 'DT_NULL', 'BT_NULL', 'DT_NULL']


***Result of Data Engineering for DroppingTime...Calculated the Time Of Journey(Top 100 records)***

In [42]:
TimeofJourney = pd.DataFrame(TOJ, columns = ["TimeOfJourney"])
print(TimeofJourney.head(5))

  TimeOfJourney
0            20
1            19
2            19
3            21
4            18


# Data Engineering on TicketAmount

In [43]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1rTNAhwzd9LpWso2_fbZWSgsuiVDbFmBI">')


In [44]:
print(BusType_result.head(5))

                             BusType  BusType_Cluster
0           Semi Sleeper cum Sleeper                6
1           Semi Sleeper cum Sleeper                6
2           Semi Sleeper cum Sleeper                6
3  Volvo B9R Multi-Axle Semi Sleeper                0
4                     Hi-Tech Non AC                0


In [0]:
PriceRange = []
BusTypeCluster = BusType_result["BusType_Cluster"].head(10000)
Source = data["Source"].head(10000)
TicketAmount = data["TicketAmount"].head(10000)

TC_df = pd.DataFrame(zip(BusTypeCluster,Source,TicketAmount),columns = ["BusType_Cluster","Source","TicketAmount"])

In [46]:
print(TC_df.head(10))

   BusType_Cluster         Source  TicketAmount
0                6      Tadipatri         600.0
1                6      Hyderabad         600.0
2                6      Hyderabad         600.0
3                0      Hyderabad           0.0
4                0      Hyderabad           0.0
5                0      Hyderabad           0.0
6                6         Guntur         600.0
7                6      Hyderabad         450.0
8                0      Hyderabad           0.0
9                0  Visakhapatnam           0.0


In [47]:
TC_df.groupby(['BusType_Cluster']).groups.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6])

In [48]:
MaxBin = TC_df.groupby(['BusType_Cluster', 'Source'])['TicketAmount'].max()
MinBin = TC_df.groupby(['BusType_Cluster', 'Source'])['TicketAmount'].min()
print(TC_df.BusType_Cluster)

0       6
1       6
2       6
3       0
4       0
5       0
6       6
7       6
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      6
16      6
17      1
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
9970    6
9971    4
9972    1
9973    3
9974    1
9975    6
9976    6
9977    0
9978    1
9979    6
9980    6
9981    1
9982    6
9983    0
9984    6
9985    1
9986    6
9987    1
9988    1
9989    1
9990    6
9991    4
9992    2
9993    6
9994    6
9995    3
9996    3
9997    2
9998    6
9999    1
Name: BusType_Cluster, Length: 10000, dtype: int64


In [0]:
maxPrice = TC_df.groupby(['BusType_Cluster', 'Source']).max()
minPrice = TC_df.groupby(['BusType_Cluster', 'Source']).min()
meanPrice = TC_df.groupby(['BusType_Cluster', 'Source']).mean()

In [50]:
gcc = TC_df.groupby(['Source', 'BusType_Cluster'])
print(gcc.first())

                               TicketAmount
Source        BusType_Cluster              
Addanki       0                         0.0
              6                       500.0
Akiveedu      1                      1560.0
              3                      2700.0
              6                       840.0
Alamuru       0                         0.0
              1                      1120.0
              6                       560.0
Alathur       2                      2000.0
Allagadda     6                       450.0
Aluva         0                         0.0
              1                      1000.0
              3                      3000.0
Amalapuram    0                         0.0
              1                      1340.0
              2                      2200.0
              3                      4020.0
              6                       670.0
Ambajipeta    1                      1220.0
              6                       610.0
Anakapalli    0                 

In [51]:
TC_df.groupby('BusType_Cluster').aggregate(['min', np.median, max])

TicketAmount                 
                         min  median      max
BusType_Cluster                              
0                        0.0     0.0    300.0
1                      940.0  1200.0   1598.0
2                     1600.0  1950.0   2600.0
3                     2650.0  3282.0   4400.0
4                     4470.0  5400.0   8000.0
5                     8400.0  9730.0  19920.0
6                      330.0   610.0    920.0

In [52]:
TC_df.groupby('BusType_Cluster').aggregate({'TicketAmount': 'min','TicketAmount': 'max'})

,TicketAmount
BusType_Cluster,
0,300.0
1,1598.0
2,2600.0
3,4400.0
4,8000.0
5,19920.0
6,920.0


Calculating the PriceRange for TicketAmount w.r.t BusType

In [53]:
"""grouped = TC_df.groupby('BusType_Cluster').agg("TicketAmount": [min, max, mean]) 
# Using ravel, and a string join, we can create better names for the columns:
#grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
"""

'grouped = TC_df.groupby(\'BusType_Cluster\').agg("TicketAmount": [min, max, mean]) \n# Using ravel, and a string join, we can create better names for the columns:\n#grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]\n'

In [54]:
result_TC_df = pd.concat([data.head(10000), pd.DataFrame(BusTypeCluster)],sort=True, columns = True)
print(result_TC_df.head(5))


TypeError: ignored

In [0]:
"""print(result_TC_df["BusTypeCluster"])
range_tc = {"500 to 1000" : 1, "1001 to 1500": 2, "1501 to 2000": 3, "2001 and more":4}
range_tc_list = []
"""


# Data Engineering on BookedDate

In [55]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1m_A1gktumTSN0M0cdog3OujQ8CPurexm">')


In [93]:
Prior_JourneyDate = []
DOJ = data["DOJ"]
BookedDate = data["BookedDate"]
print(DOJ.head(5))
print(BookedDate.head(5))

0    2014-10-06
1    2014-10-02
2    2014-10-04
3    2015-01-18
4    2015-01-18
Name: DOJ, dtype: object
0    2015-06-20
1    2015-06-20
2    2015-06-20
3    2015-01-09
4    2015-01-13
Name: BookedDate, dtype: object


In [0]:
from datetime import datetime
date_format = "%Y-%m-%d"

for BD, DJ in zip(BookedDate, DOJ):
  a = datetime.strptime(BD, date_format)
  b = datetime.strptime(DJ, date_format)
  #print("A",a); print("B",b);print("Diff",diff)
  Prior_JourneyDate.append(str(b - a))

In [96]:
print(Prior_JourneyDate[0:10])

['-257 days, 0:00:00', '-261 days, 0:00:00', '-259 days, 0:00:00', '9 days, 0:00:00', '5 days, 0:00:00', '5 days, 0:00:00', '-161 days, 0:00:00', '-138 days, 0:00:00', '1 day, 0:00:00', '1 day, 0:00:00']


***Result for Data Engineering on BookedDate***

In [98]:
BookedDate_result = pd.DataFrame(zip(DOJ, BookedDate,Prior_JourneyDate),columns =["Date Of Journey","Booked Date", "Diffrence"])
print(BookedDate_result.head(10))

  Date Of Journey Booked Date           Diffrence
0      2014-10-06  2015-06-20  -257 days, 0:00:00
1      2014-10-02  2015-06-20  -261 days, 0:00:00
2      2014-10-04  2015-06-20  -259 days, 0:00:00
3      2015-01-18  2015-01-09     9 days, 0:00:00
4      2015-01-18  2015-01-13     5 days, 0:00:00
5      2015-01-23  2015-01-18     5 days, 0:00:00
6      2015-01-19  2015-06-29  -161 days, 0:00:00
7      2015-01-22  2015-06-09  -138 days, 0:00:00
8      2015-01-23  2015-01-22      1 day, 0:00:00
9      2015-01-24  2015-01-23      1 day, 0:00:00


# Data Engineering on CancellationDate

In [99]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1a3NMjacLT6R6K9m8SznoYk9trp5hlDwI">')


In [103]:
#Putting Cancellation Flag
import math
Cancellation_Pool = []
CancellationDate = data["CancellationDate"]
for cancel in Cancellation:
    if type(cancel) is float:
        if math.isnan(cancel):
            Cancellation_Pool.append("0")
    else:
        Cancellation_Pool.append("1")
print("Cancellation_Pool",Cancellation_Pool[:10])       

Cancellation_Pool ['0', '0', '0', '1', '1', '1', '0', '0', '1', '1']


In [0]:
DOJ = data["DOJ"]
BookedDate = data["BookedDate"]


In [114]:
# Number of Days before DOJ
from datetime import datetime
date_format = "%Y-%m-%d"
cancel_before_DOJ = []
for DJ, CD in zip(DOJ, CancellationDate):
  a = datetime.strptime(BD, date_format)
  b = datetime.strptime(DJ, date_format)
  #print("A",a); print("B",b);print("Diff",diff)
  cancel_before_DOJ.append(str(b - a))
print(cancel_before_DOJ[0:10])

['-267 days, 0:00:00', '-271 days, 0:00:00', '-269 days, 0:00:00', '-163 days, 0:00:00', '-163 days, 0:00:00', '-158 days, 0:00:00', '-162 days, 0:00:00', '-159 days, 0:00:00', '-158 days, 0:00:00', '-157 days, 0:00:00']


In [115]:
# Number of Days after Booking
cancel_after_booking = []
for BD, CD in zip(BookedDate, CancellationDate):
  a = datetime.strptime(BD, date_format)
  b = datetime.strptime(DJ, date_format)
  #print("A",a); print("B",b);print("Diff",diff)
  cancel_after_booking.append(str(a - b))
print(cancel_after_booking[:10])

['-27 days, 0:00:00', '-27 days, 0:00:00', '-27 days, 0:00:00', '-189 days, 0:00:00', '-185 days, 0:00:00', '-180 days, 0:00:00', '-18 days, 0:00:00', '-38 days, 0:00:00', '-176 days, 0:00:00', '-175 days, 0:00:00']


***Result of Data Engieering for CancellationDate***

In [116]:
# Result
CancellationDate_result = pd.DataFrame(zip(DOJ, BookedDate, CancellationDate, Cancellation_Pool,cancel_before_DOJ,cancel_after_booking), columns = ["DOJ","Booked Date","Cancellation Data","Cancellation Pool","Cancellation w.r.t DOJ","Cancellation w.r.t Booking Date"])
print(CancellationDate_result.head(5))

          DOJ  ... Cancellation w.r.t Booking Date
0  2014-10-06  ...               -27 days, 0:00:00
1  2014-10-02  ...               -27 days, 0:00:00
2  2014-10-04  ...               -27 days, 0:00:00
3  2015-01-18  ...              -189 days, 0:00:00
4  2015-01-18  ...              -185 days, 0:00:00

[5 rows x 6 columns]


# Data Engineering on APISource


In [117]:
import IPython
IPython.display.HTML('<img src="https://drive.google.com/uc?id=1U11uxR1D-jc2I-Pk-cQ-I2F4Ly7ggSmV">')


In [122]:
# Getting Column Data
APISource = data["APISource"]
print(APISource[0:10])

0           Diwakar Travels
1           Diwakar Travels
2           Diwakar Travels
3            Kaveri Travels
4            Kaveri Travels
5              VRCR Travels
6    Sree Kaleswari Travels
7           Diwakar Travels
8              VRCR Travels
9            Kaveri Travels
Name: APISource, dtype: object


In [129]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df
  
result = handle_non_numerical_data(pd.DataFrame(APISource,columns=["APISource"]))


   APISource
0         11
1         11
2         11
3         58
4         58
5         67
6         28
7         11
8         67
9         58
<class 'pandas.core.frame.DataFrame'>


***Result for Data Engineering for APISource Columns***

In [138]:
APISOurceID =result.APISource
APISource_result = pd.DataFrame(zip(APISource,APISOurceID), columns = ["APISource","APISOurceID"])

print(APISource_result.head())

         APISource  APISOurceID
0  Diwakar Travels           11
1  Diwakar Travels           11
2  Diwakar Travels           11
3   Kaveri Travels           58
4   Kaveri Travels           58
